In [ ]:
# Library📃

import glob, warnings, polars as pl, datetime, sqlite3, time, os, zipfile, xml.dom.minidom
from datetime import datetime as dt, time as t, timedelta, date
import pandas as pd, numpy as np, sqlalchemy as sa, xlsxwriter
from sqlalchemy import create_engine
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

# Link📃
Link_DB = os.path.join(os.environ['USERPROFILE'], r'Desktop//Bcom_DB.db')
conn = create_engine(f"sqlite:///{Link_DB}")
pattern_raw_path=os.path.join(os.environ['USERPROFILE'], r'Concentrix Corporation\CNXVN - WFM Team - Documents\DataBase\DataRaw\BKN\SCHEDULE_MAPPING\PatternID')
special_request_path=os.path.join(os.environ['USERPROFILE'], r'Concentrix Corporation\CNXVN - WFM Team - Documents\DataBase\DataRaw\BKN\SCHEDULE_MAPPING\Special Request')
schedule_path=os.path.join(os.environ['USERPROFILE'], r'Concentrix Corporation\CNXVN - WFM Team - Documents\DataBase\DataFrame\BKN\SCHEDULE')
team_shift_path=os.path.join(os.environ['USERPROFILE'], r'Concentrix Corporation\CNXVN - WFM Team - Documents\DataBase\DataRaw\BKN\SCHEDULE_MAPPING\Team shift')
ews_path=os.path.join(os.environ['USERPROFILE'], r'Concentrix Corporation\CNXVN - WFM Team - Documents\DataBase\DataRaw\BKN\WFMForm')
roster_path=os.path.join(os.environ['USERPROFILE'], r'Concentrix Corporation\CNXVN - WFM Team - Documents\DataBase\DataRaw\BKN\ROSTER\Schedule Query\*.xlsx')
agents_path=os.path.join(os.environ['USERPROFILE'], r'Concentrix Corporation\CNXVN - WFM Team - Documents\DataBase\DataRaw\BKN\AGENTS')

# Weekoff Rule📃
weekoff_mapping = {
    'MonTue': ('MonTue','TueWed', 'MonSun'),
    'TueWed': ('MonTue','TueWed', 'WedThu','MonSun'),
    'WedThu': ('MonTue','TueWed', 'WedThu','ThuFri','MonSun'),
    'ThuFri': ('MonTue','TueWed', 'WedThu','ThuFri', 'FriSat','MonSun'),
    'FriSat': ('MonTue','TueWed', 'WedThu','ThuFri', 'FriSat','SatSun','MonSun'),
    'SatSun': ('MonTue','TueWed', 'WedThu','ThuFri', 'FriSat','SatSun','MonSun'),
    'MonSun': ('MonTue','TueWed', 'WedThu','ThuFri', 'FriSat','SatSun','MonSun')
}
weekoff_mapping_pre = {
    'MonTue': ('MonTue','TueWed', 'WedThu','ThuFri', 'FriSat','SatSun','MonSun'),
    'TueWed': ('MonTue','TueWed', 'WedThu','ThuFri', 'FriSat','SatSun','MonSun'),
    'WedThu': ('TueWed', 'WedThu','ThuFri', 'FriSat','SatSun','MonSun'),
    'ThuFri': ('WedThu','ThuFri', 'FriSat','SatSun','MonSun'),
    'FriSat': ('ThuFri', 'FriSat','SatSun','MonSun'),
    'SatSun': ('FriSat','SatSun','MonSun'),
    'MonSun': ('MonTue','TueWed', 'WedThu','ThuFri', 'FriSat','SatSun','MonSun')    
}
# Nextweek first day📃
today = datetime.date.today()
next_week = pd.to_datetime(today) + pd.offsets.Week(weekday=0)
next_week = next_week.date()
print(next_week)
current_week = pd.to_datetime(today) + pd.offsets.Week(weekday=0) - pd.Timedelta(days=1)
current_week = current_week.date()
print(current_week)

In [ ]:
# Roster & Pattern 🧩

current_week = pd.to_datetime(today) + pd.offsets.Week(weekday=0) - pd.Timedelta(days=1)
current_week = current_week.date()
Data_Schedule = []
for file in glob.glob(roster_path):
    filename = os.path.basename(file)
    Read = pl.read_excel(file, infer_schema_length=None, engine="calamine", sheet_name="Sheet1")
    Schedule = Read.select(pl.all(), pl.lit(filename).alias("Filename"))
    Data_Schedule.append(Schedule)
roster_df = pl.concat(Data_Schedule, how="diagonal")
roster_df =roster_df.to_pandas()
roster_df['Attribute']=pd.to_datetime(roster_df['Attribute'],format='mixed').dt.date
roster_df =roster_df.loc[((roster_df['Attribute']==current_week) & (roster_df['week_shift'] !='Training') & (roster_df['week_shift'] !=''))]
roster_df["ShiftStart"] = pd.to_datetime(roster_df['Attribute'].astype(str) + " " + roster_df["week_shift"].str[:2] + ":00:00", format='%Y-%m-%d %H:%M:%S')
roster_df["ShiftEnd"] = roster_df["ShiftStart"] + pd.Timedelta(hours=9)
roster_df =roster_df[['Emp ID','week_off','LOB','team_leader',"ShiftStart","ShiftEnd",'week_shift']]
roster_df=roster_df.rename(columns={'Emp ID':'EID','week_off':'WeekOff'})

# Merge with team shift to get shift type💡
team_shift = pd.read_excel(f"{team_shift_path}/Team shift.xlsx", sheet_name="Sheet1")
roster_df = pd.merge(roster_df, team_shift, left_on='team_leader', right_on='EID', how='left')
roster_df['Prioritized shift'] = roster_df['SHIFT'].apply(lambda x: 'NS' if x == 'NS' else 'DS')
roster_df = roster_df.drop(columns=['EID_y', 'TL']).rename(columns={'EID_x': 'EID'})
# Merge ews to remove attrition💡
ews = pd.read_excel(f"{ews_path}/WFMForm.xlsx", sheet_name="EWSView", dtype={"EID": str})
roster_df["EID"] = roster_df["EID"].astype(str)
roster_df = pd.merge(roster_df, ews[["EID", "LWD_OnFloor"]], on="EID", how="left")
roster_df["LWD_OnFloor"] = roster_df["LWD_OnFloor"].fillna("2030-01-01")
roster_df["LWD_OnFloor"] = pd.to_datetime(roster_df["LWD_OnFloor"], format="mixed").dt.date
roster_df["LWD_OnFloor"] = (roster_df["LWD_OnFloor"] > next_week).astype(int)  
roster_df = roster_df[roster_df["LWD_OnFloor"] == 1]  
roster_df[["ShiftStart", "ShiftEnd"]] = roster_df[["ShiftStart", "ShiftEnd"]].apply(pd.to_datetime, format="mixed")  
roster_df["Gap"] = roster_df["ShiftEnd"] + pd.Timedelta(hours=12)
roster_df["Order"] = roster_df["WeekOff"].isin(("MonSun", "SatSun")).astype(int) 
roster_df.sort_values(by=['Gap', 'team_leader'], ascending=True, inplace=True)
roster_df.sort_values(by=['Order'], ascending=False, inplace=True)
roster_df.sort_values(by=['Prioritized shift'], ascending=False, inplace=True)
roster_df.drop(columns=[ "Order", "SHIFT", "LWD_OnFloor"], inplace=True)

# Create Pattern
next_week = next_week.strftime("%Y%m%d")
path=".xlsx"
filename= next_week+path
pattern_df = pd.read_excel(f"{pattern_raw_path}/{filename}", sheet_name="Sheet1")
ds_shifts = ('0000-0900', '0100-1000', '0200-1100', '0300-1200', '0400-1300', '0500-1400', '0600-1500', '0700-1600', 
             '0800-1700', '0900-1800', '1000-1900', '1100-2000', '1200-2100', '1300-2200', '1400-2300', '1500-0000')
ns_shifts = ('1600-0100', '1700-0200', '1800-0300', '1900-0400', '2000-0500', '2100-0600', '2200-0700', '2300-0800')
pattern_df['ShiftType'] = pattern_df['Shift'].apply(lambda x: 'NS' if x in ns_shifts else 'DS')
pattern_df["ShiftStart"] = pd.to_datetime(pattern_df['Date'].astype(str) + " " + pattern_df["Shift"].str[:2] + ":00:00", format='%Y-%m-%d %H:%M:%S')
pattern_df["ShiftEnd"] = pattern_df["ShiftStart"] + pd.Timedelta(hours=9)
pattern_df["Date"] = pd.to_datetime(pattern_df["Date"]).dt.date
pattern_df['Check'] = pattern_df.apply(lambda x: 0 if x['RD'] in ('MonSun', 'MonTue') else 1, axis=1)
pattern_df = pattern_df.sort_values(by=['ShiftType', 'Check', 'ShiftStart'], ascending=False)
pattern_df['EID'] = pattern_df['EID'].fillna('NA')
pattern_df['ID Pattern'] = pattern_df['ID Pattern'].astype(str)
pattern_df['Check'] = pattern_df['LOB'] + pattern_df['ID Pattern']
pattern_df = pattern_df[['Date', 'LOB', 'ID Pattern', 'ShiftType', 'RD', 'Shift', 'ShiftStart', 'ShiftEnd', 'EID', 'Check']]

print(roster_df)
print(pattern_df)

In [ ]:
# Request ✨
next_week = pd.to_datetime(today) + pd.offsets.Week(weekday=0)
next_week = next_week.date()

special_request_df = pd.read_excel(special_request_path+"/"+filename,sheet_name="Sheet1")
special_request_df=special_request_df.fillna(0)
special_request_df["ShiftStart"] = special_request_df.apply(lambda x: x["Shift"][0:2] if x['Shift']!=0 else x['Shift'], axis=1)
special_request_df["ShiftStart"]=special_request_df["ShiftStart"].astype("int64")
special_request_df["ShiftStart"]=pd.to_datetime(special_request_df["ShiftStart"],format='%H').dt.time
special_request_df["ShiftStart"]=pd.to_datetime(special_request_df['Date'].astype(str) + special_request_df["ShiftStart"].astype(str), format = '%Y-%m-%d%H:%M:%S')
special_request_df["ShiftEnd"]=pd.to_datetime(special_request_df["ShiftStart"],format='mixed') + pd.Timedelta(hours=9)
special_request_df["Date"]=pd.to_datetime(special_request_df["Date"],format='mixed').dt.date
special_request_df["EID"]=special_request_df["EID"].astype("str")
special_request_df=pd.merge(special_request_df,roster_df[['LOB','EID','WeekOff','Gap']], left_on='EID', right_on='EID', how='left')
special_request_df['LOB']=special_request_df.apply(lambda x: x['LOB_y'] if x['LOB_y'] in ('VICSG','VICSP','Senior VICSP') else x['LOB_x'], axis=1)
special_request_df['WeekOff']=special_request_df.apply(lambda x: x['WeekOff_y'] if x['WeekOff_x'] ==0 else x['WeekOff_x'], axis=1)
special_request_df = pd.merge(special_request_df, ews[["EID", "LWD_OnFloor"]], on="EID", how="left")
special_request_df["LWD_OnFloor"] = special_request_df["LWD_OnFloor"].fillna("2030-01-01")
special_request_df["LWD_OnFloor"] = pd.to_datetime(special_request_df["LWD_OnFloor"], format="mixed").dt.date
special_request_df["LWD_OnFloor"] = (special_request_df["LWD_OnFloor"] > next_week).astype(int) 
special_request_df = special_request_df[special_request_df["LWD_OnFloor"] == 1] 
special_request_df=special_request_df.drop(columns=['LOB_x', 'LOB_y','WeekOff_x','WeekOff_y',"LWD_OnFloor"])

# Special request mapping
schedule_sr=special_request_df
def tim_emp_phu_hop(EID, rd,shifttype, shift, lob, gap,pattern_df):
    # Define the condition for the filter
    lob_condition = ((pattern_df['LOB'].isin(['VICSG', 'VICSP', 'Senior VICSP'])) & (lob == pattern_df['LOB'])) | \
                    ((lob not in ['VICSG', 'VICSP', 'Senior VICSP']) & (~pattern_df['LOB'].isin(['VICSG', 'VICSP', 'Senior VICSP'])))
    gap_hour = gap.hour
    # Filter rows based on the parameters passed
    filtered_roster = pattern_df[
        pattern_df['RD'].isin(weekoff_mapping_pre) &
        pattern_df['RD'].apply(lambda x: rd in weekoff_mapping.get(x, [])) &
        ((pattern_df['ShiftStart'].dt.hour > gap_hour) | 
        (pattern_df['RD'].isin(['MonTue','MonSun'])) | (rd in ('SatSun', 'MonSun'))) &
        ((rd == 0) | (pattern_df['RD'] == rd)) & 
        ((lob == 0) | (pattern_df['LOB'] == lob)) & 
        ((shift == 0) | (pattern_df['Shift'] == shift)) & ((shifttype == 0) | (pattern_df['ShiftType'] == shifttype)) &
        lob_condition]
    # Return the filtered 'Check' column as a list
    return filtered_roster['Check'].tolist() if not filtered_roster.empty else []
schedule_sr['emp_list'] = schedule_sr.apply(lambda row: tim_emp_phu_hop(row['EID'], row['WeekOff'], row['ShiftType'], row['Shift'], row['LOB'],row['Gap'], pattern_df), axis=1)
processed_employees = []
schedule_sr['ID Pattern'] = None
for i, row in schedule_sr.iterrows():
   for emp_list in row['emp_list']:
           if emp_list not in processed_employees:                   
                schedule_sr.loc[i, 'ID Pattern'] = emp_list
                processed_employees.append(emp_list)
                break
schedule_sr['ID Pattern']=schedule_sr['ID Pattern'].fillna(0)

# Loai cac ID pattern da xep cho Special request ra khoi pattern_df
pattern_df=pd.merge(pattern_df,schedule_sr[['EID','ID Pattern']], left_on='Check', right_on='ID Pattern', how='left')
pattern_df['EID_y']=pattern_df['EID_y'].fillna(0)
pattern_df['EID']=pattern_df.apply(lambda x: x['EID_y'] if x['EID_y']!=0 else x['EID_x'], axis=1)
pattern_df=pattern_df.drop(columns=['EID_x', 'EID_y', 'ID Pattern_y','Check'])
pattern_df=pattern_df.rename(columns={'ID Pattern_x':'ID Pattern'})
pattern_df['EID']=pattern_df['EID'].astype("str")

print(pattern_df)

In [ ]:
# Mapping 📥

roster=pd.merge(roster_df,pattern_df[['EID','RD']], left_on='EID', right_on='EID', how='left')
roster=roster.loc[(roster['RD'].isnull()==True)]
roster=roster.drop(columns=['RD'])
schedule=pattern_df
pd.set_option('future.no_silent_downcasting', True)
def tim_emp_phu_hop(EID, rd, shiftype, shiftstart, lob, roster):
    if EID == 'NA':
        shiftstart_hour = shiftstart.hour
        shiftstart_minute = shiftstart.minute
        # Filter
        filtered_roster = roster[
            roster['WeekOff'].isin(weekoff_mapping) &
            roster['WeekOff'].apply(lambda x: rd in weekoff_mapping.get(x, [])) &
            ((roster['Gap'].dt.hour < shiftstart_hour) | 
             ((roster['Gap'].dt.hour == shiftstart_hour) & (roster['Gap'].dt.minute <= shiftstart_minute)) |
             (roster['WeekOff'].isin(['SatSun', 'MonSun'])) | (rd in ('MonTue','MonSun'))) &
            (lob == roster['LOB']) & (roster['Prioritized shift']==shiftype)
            ]
        return filtered_roster['EID'].tolist()
    else:
        return [] 
schedule['emp_list'] = schedule.apply(lambda row: tim_emp_phu_hop(row['EID'], row['RD'], row['ShiftType'], row['ShiftStart'], row['LOB'], roster), axis=1)
processed_employees = []
schedule['EID_2'] = None
# Loop 1
for i, row in schedule.iterrows():
   for emp_list in row['emp_list']:
           if emp_list not in processed_employees:                   
                schedule.loc[i, 'EID_2'] = emp_list
                processed_employees.append(emp_list)
                break
schedule['EID_2'] = schedule['EID_2'].fillna(0)
schedule['EID']=schedule.apply(lambda x: x['EID_2'] if x['EID_2'] !=0 else x['EID'], axis=1)
#Loop 2
for _ in range(4): 
    schedule['EID_2'] = None
    for i, row in schedule.iterrows():
       for emp_list in row['emp_list']:
               if emp_list not in processed_employees:
                   if schedule.loc[i,'EID']=='NA':
                        schedule.loc[i, 'EID_2'] = emp_list
                        processed_employees.append(emp_list)
                        break
    schedule['EID_2'] = schedule['EID_2'].fillna(0)
    schedule['EID']=schedule.apply(lambda x: x['EID_2'] if x['EID_2'] !=0 else x['EID'], axis=1)
# Map with diff LOB
def tim_emp_phu_hop(EID, rd,shiftype, shiftstart, lob, roster):
    if EID == 'NA':
        shiftstart_hour = shiftstart.hour
        shiftstart_minute = shiftstart.minute
        # Filter
        filtered_roster = roster[
            roster['WeekOff'].isin(weekoff_mapping) &
            roster['WeekOff'].apply(lambda x: rd in weekoff_mapping.get(x, [])) &
            ((roster['Gap'].dt.hour < shiftstart_hour) | 
             ((roster['Gap'].dt.hour == shiftstart_hour) & (roster['Gap'].dt.minute <= shiftstart_minute)) |
             (roster['WeekOff'].isin(['SatSun', 'MonSun'])) | (rd in ('MonTue','MonSun'))) &
            (((roster['LOB'].isin(['VICSG', 'VICSP', 'Senior VICSP'])) & (lob == roster['LOB'])) |
             ((lob not in (['VICSG', 'VICSP', 'Senior VICSP'])) & (~roster['LOB'].isin (['VICSG', 'VICSP', 'Senior VICSP']))))
            & (roster['Prioritized shift']==shiftype)
            ]
        return filtered_roster['EID'].tolist()
    else:
        return [] 
schedule['emp_list'] = schedule.apply(lambda row: tim_emp_phu_hop(row['EID'], row['RD'],row['ShiftType'], row['ShiftStart'], row['LOB'], roster), axis=1)
# Loop 1 with diff LOB
for i, row in schedule.iterrows():
   for emp_list in row['emp_list']:
           if emp_list not in processed_employees:                   
                schedule.loc[i, 'EID_2'] = emp_list
                processed_employees.append(emp_list)
                break
schedule['EID_2'] = schedule['EID_2'].fillna(0)
schedule['EID']=schedule.apply(lambda x: x['EID_2'] if x['EID_2'] !=0 else x['EID'], axis=1)
#Loop 2 with diff LOB
for _ in range(4): 
    schedule['EID_2'] = None
    for i, row in schedule.iterrows():
       for emp_list in row['emp_list']:
               if emp_list not in processed_employees:
                   if schedule.loc[i,'EID']=='NA':
                        schedule.loc[i, 'EID_2'] = emp_list
                        processed_employees.append(emp_list)
                        break
    schedule['EID_2'] = schedule['EID_2'].fillna(0)
    schedule['EID']=schedule.apply(lambda x: x['EID_2'] if x['EID_2'] !=0 else x['EID'], axis=1)

print(schedule)

In [ ]:
# Out put 💡

# unused pattern
pattern = schedule[['Date', 'LOB', 'ID Pattern', 'ShiftType', 'RD', 'Shift', 'ShiftStart', 'ShiftEnd', 'EID']]
unused_pattern = pattern[pattern['EID'] == 'NA']
# unscheduled agent
unschedule_agent = pd.merge(roster_df, schedule[['ID Pattern', 'EID']], on='EID', how='left')
unschedule_agent = unschedule_agent[unschedule_agent['ID Pattern'].isna()]
# Function to write DataFrames to Excel
def write_to_excel(filename, data_dict):
    with pd.ExcelWriter(filename, engine='xlsxwriter') as writer:
        for sheet, data in data_dict.items():
            data.to_excel(writer, sheet_name=sheet, index=False)
# Prepare data and write to Excel
schedule_data = {'PATTERN': pattern, 'UNUSED_PATTERN': unused_pattern, 'UNSCHEDULE_AGENT': unschedule_agent,'ROSTER': roster_df}
write_to_excel(f"{schedule_path}/Schedule_{filename}", schedule_data)

# Print the data
print(pattern)
print(unused_pattern)
print(unschedule_agent)